# Final Project
Abigail Allen  
Josh Urry  
Trevor Jex

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import HeatMap
%matplotlib inline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score

In [ ]:
# Load data
df = pd.read_csv("Motor_Vehicle_Crashes_-_Case_Information__Three_Year_Window.csv")

In [ ]:
df.head()

In [ ]:
# Check number of samples
len(df)

## Preprocessing

In [ ]:
# Check NA values
df.isna().sum()

Let's drop the "DOT Reference Marker Location" variable, since the majority of accidents have this missing. Maybe we could just remove the rows where the Municipality is blank. Or just drop it as well and just use the County.

In [ ]:
df.drop("DOT Reference Marker Location", axis = 1, inplace = True)

Let's figure out how to handle the date variables.

In [ ]:
df.Year.unique()

We could represent these as 0, 1, 2, 3, and 4 as a variable that represents "years from 2017", so it is numeric.

In [ ]:
# It may not make a big difference if we don't do this. It makes the charts confusing so let's keep it for now.
# df.Year.replace({2017: 0, 2018: 1, 2019: 2, 2020: 3, 2021: 4}, inplace = True)

Let's convert time and date to date time objects and extract the minutes, hours, month, and day

In [ ]:
df.Time = pd.to_datetime(df.Time)
df.Date = pd.to_datetime(df.Date)

In [ ]:
df["month"] = df.Date.dt.month
df["day"] = df.Date.dt.day
df["hour"] = df.Time.dt.hour
df["minute"] = df.Time.dt.minute # My guess is that we'll end up dropping this

In [ ]:
# Drop Date and Time variables 
# df.drop(["Date", "Time"], axis = 1, inplace = True)
df.drop(["Date"], axis = 1, inplace = True)

In [ ]:
# Now let's convert day of the week to numeric
df["Day of Week"].replace({"Sunday": 0, "Monday": 1, "Tuesday": 2, "Wednesday": 3, "Thursday": 4,
                          "Friday": 5, "Saturday": 6}, inplace = True)

In [ ]:
df.head()

Let's figure out the target variable

In [ ]:
df["Event Descriptor"].unique()

In [ ]:
df["Crash Descriptor"].unique()

We wil use the crash descriptor variable as the outcome.

## Data Visualization

In [ ]:
# Lineplot of accidents over time
df.groupby('Year')['Time'].count().plot()
plt.xlabel('Year')
plt.ylabel('Number of Accidents')
plt.title('Number of Accidents over Time')
plt.show()

In [ ]:
# Barplot of accidents per year
counts = df['Year'].value_counts().sort_index()
counts.plot.bar()
plt.xlabel('Year')
plt.ylabel('Count')
plt.title('Accidents per Year')
plt.show()

In [ ]:
# Barplot of accidents per week
counts = df['Day of Week'].value_counts().sort_index()
counts.plot.bar()
plt.xlabel('Week Day')
plt.ylabel('Count')
plt.title('Accidents per Week Day')
plt.show()

In [ ]:
# Barplot of accidents per week
counts = df['month'].value_counts().sort_index()
counts.plot.line()
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Accidents per Month')
plt.show()

In [ ]:
# Barplot of accidents per week, excluding 2020
counts = df[df['Year'] != 2020]['month'].value_counts().sort_index()
counts.plot.line()
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Accidents per Month (excluding 2020)')
plt.show()

In [ ]:
df_grouped = df.groupby('hour').count()
plt.plot(df_grouped.index, df_grouped['Number of Vehicles Involved'])

plt.xlabel('Hour of Day')
plt.ylabel('Number of Accidents')
plt.title('Accidents by Hour of the Day')
plt.show()

In [ ]:
# Barplot of top 10 accidents per county
counts = df['County Name'].value_counts().head(10)
counts.plot.bar()
plt.xlabel('County')
plt.ylabel('Count')
plt.title('Accidents per County')
plt.xticks(rotation=60, ha='right')
plt.show()

In [ ]:
# Barplot of top 10 event decriptors
counts = df['Event Descriptor'].value_counts().head(10)
counts.plot.bar()
plt.xlabel('Event Descriptor')
plt.ylabel('Count')
plt.title('Event Descriptor Counts')
plt.show()

In [ ]:
# Barplot of about of accident descriptors
counts = df['Crash Descriptor'].value_counts()
counts.plot.bar()
plt.xlabel('Accident Descriptor')
plt.ylabel('Count')
plt.title('Accident Descriptor Counts')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Stacked barchart of lighting, weather, road surface, and traffice control device per accident description
conditions = ['Lighting Conditions', 'Weather Conditions', 'Road Surface Conditions']

# Create a list of pivot tables and charts for each condition
for condition in conditions:
    pivot_table = df.pivot_table(index=condition, columns='Collision Type Descriptor', values='Year', aggfunc='count')
    ax = pivot_table.plot(kind='bar', stacked=True, figsize=(10, 6))

    ax.set_title(f'Distribution of Crash Types by {condition}')
    ax.set_xlabel(condition)
    ax.set_ylabel('Number of Accidents')
    plt.show()

In [ ]:
# Drop Time
df.drop("Time", axis = 1 , inplace = True)

## Feature Extraction

One hot encoding all of the categorical variables leaves us with a lot of features. Let's see if we can do some feature extraction to whittle that down. Let's start with the event descriptor variable.

In [ ]:
df["Event Descriptor"].unique()

As shown above, there are alot of categories for this one. The visualizations also show that there are relatively few occurances besides collision with another car. Let's just recode them as "Collision with Fixed Object", "Collision with Other Vehicle", "Collision with Pedestrian/Animal", and "Non-Collision"

In [ ]:
# Define conditions to recode
conditions = [
    # Conditions for "Collision with Fixed Object"
    ((df["Event Descriptor"] == "Earth Embankment/Rock Cut/Ditch, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Fire Hydrant, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Other Fixed Object*, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Barrier, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Curbing, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Building/Wall, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Light Support/Utility Pole, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Median - Not At End, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Tree, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Guide Rail - Not At End, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Guide Rail - End, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Bridge Structure, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Snow Embankment, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Sign Post, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Culver/Head Wall, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Fence, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Crash Cushion, Collision With Fixed Object") |
    (df["Event Descriptor"] == "Median - End, Collision With Fixed Object")),
    # Conditions for "Collision with Other Vehicle (I'm going to count trains in here too)"
    ((df["Event Descriptor"] == "Other Motor Vehicle, Collision With") |
    (df["Event Descriptor"] == "Railroad Train, Collision With")),
    # Conditions for "Collision with Pedestrian/Animal"
    ((df["Event Descriptor"] == "Pedestrian, Collision With") | 
     (df["Event Descriptor"] == "Bicyclist, Collision With") |
     (df["Event Descriptor"] == "Deer") | (df["Event Descriptor"] == "Animal, Collision With") | 
     (df["Event Descriptor"] == "Other Pedestrian")| (df["Event Descriptor"] == "In-Line Skater, Collision With")|
     (df["Event Descriptor"] == "Other Object (Not Fixed)*, Collision With")),
    # Conditions for "Non-Collision"
    ((df["Event Descriptor"] == "Overturned, Non-Collision") | (df["Event Descriptor"] == "Other*, Non-Collision") |
     (df["Event Descriptor"] == "Ran Off Roadway Only, Non-Collision") |
     (df["Event Descriptor"] == "Submersion, Non-Collision") | 
     (df["Event Descriptor"] == "Fire/Explosion, Non-Collision"))
]

# Corresponding values with the four conditions
value = [
    "Collision with Fixed Object",
    "Collision with Other Vehicle",
    "Collision with Pedestrian/Animal",
    "Non-Collision"
    ]

# Recode
df["Event Descriptor"] = np.select(conditions, value)

In [ ]:
df["Event Descriptor"].unique()

We may lose some information here, but it is much more manageable. Let's look at collision type descriptor next.

In [ ]:
df["Collision Type Descriptor"].unique()

In [ ]:
# Look at frequencies of co-occurences
ctd_frequencies = pd.crosstab(df["Collision Type Descriptor"], df["Crash Descriptor"], normalize = "index")
ctd_frequencies

I'm not quite sure what the difference is between the different left/right turn variables. There seem to be some minor differences above, but nothing substantial. Let's combine those.

In [ ]:
df["Collision Type Descriptor"].replace({"LEFT TURN (3)": "LEFT TURN", "LEFT TURN (0)": "LEFT TURN",
                                        "RIGHT TURN (6)": "RIGHT TURN", "RIGHT TURN (5)": "RIGHT TURN"}, inplace = True)

In [ ]:
df["Collision Type Descriptor"].unique()

Lighting and Road Descriptor look fine. I'm thinking we may only need one of the road surface or weather condition variables. Let's check their mutual information.

In [ ]:
df["Road Surface Conditions"].unique()

In [ ]:
df["Weather Conditions"].unique()

In [ ]:
# Look at frequencies of co-occurences
cond_frequencies = pd.crosstab(df["Road Surface Conditions"], df["Weather Conditions"], normalize = "index")
cond_frequencies

In [ ]:
# Look at mutual info
mutual_info_score(df["Road Surface Conditions"], df["Weather Conditions"])

It looks like these are really related. Let's just stick with road contitions, since I feel that is more applicable here.

In [ ]:
df.drop("Weather Conditions", axis = 1, inplace = True)

Now, let's look at Traffic Control Devices

In [ ]:
df["Traffic Control Device"].value_counts()

I'm not convinced this will be very informative. Check crosstabs.

In [ ]:
# Look at frequencies of co-occurences
dev_frequencies = pd.crosstab(df["Traffic Control Device"], df["Crash Descriptor"], normalize = "index")
dev_frequencies

It looks like some features would potentially be useful in distinguishing some classes, like Officer/Guard for injury, or Police/Fire Emergency for property and injury, but overall there really doesn't seem to be that much variation between these. Let's drop for now. We can always add it in again.

In [ ]:
df.drop("Traffic Control Device", axis = 1, inplace = True)

Now, Police Report

In [ ]:
df["Police Report"].unique()

We can just recode this to a binary variable now, so we don't have a column we don't need from one hot encoding.

In [ ]:
df["Police Report"].replace({"Y": 1, "N": 0}, inplace = True)

Finally, let's look at Pedestrian Bicyclist Action. Let's not worry about Municipality or County for now.

In [ ]:
df["Pedestrian Bicyclist Action"].value_counts()

These realatively do not happen that often. Let's drop for now.

In [ ]:
df.head()

In [ ]:
Xtransformer = ColumnTransformer(
[
    ('categorical', OneHotEncoder(), ['Collision Type Descriptor', 'Lighting Conditions', 'Road Descriptor', 
                                      'Road Surface Conditions', 'Event Descriptor']),
    ('drop', 'drop', ['Municipality', 'County Name', 'Pedestrian Bicyclist Action', 'Crash Descriptor'])   
],
    remainder='passthrough'
)

# We shouldn't one hot encode the outcome. That won't work with predictive modeling. We could code the values as 1-4
# Ytransformer = ColumnTransformer(
#     [
#         ('categorical', OneHotEncoder(), ['Crash Descriptor'])
#     ],
#     remainder='drop'
# )

X = Xtransformer.fit_transform(df)
# y = Ytransformer.fit_transform(df)
y = df[["Crash Descriptor"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X_train.shape

## PCA

In [ ]:
# Standardize the data
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)

In [ ]:
# Perform PCA to get projected values
pca = PCA().fit(X_train)
projected_X_train = pca.transform(X_train)

In [ ]:
# Show explained variance
pca.explained_variance_ratio_

Plot the explained variance and number of PCs. Check for 90%, 95%, and 99% of variance

In [ ]:
# get total explained variance ratio
y_axis = np.cumsum(pca.explained_variance_ratio_)
# Get number of pcs
x_axis = np.arange(1, len(y_axis) + 1, step = 1)

# Now plot it
fig = plt.figure(figsize = (12,6))
pcs = sns.lineplot(x = x_axis, y = y_axis, markers = True, color = "grey", marker = "o", dashes=[2])
# Add lines signifying cutoffs
plt.axhline(y=0.90, color='b', linestyle='-', label = "90%")
plt.axhline(y=0.95, color='orange', linestyle='-', label = "95%")
plt.axhline(y=0.99, color='r', linestyle='-', label = "99%")

pcs.set(xlabel = "Number of Components", ylabel = "Cumulative Variance Percent", 
        title = "Number of Components and Explained Variance",
        xticks = x_axis)
plt.legend()

Looks like we need 3 pcs to capture both 90% and 95% of the variance, and 6 if we want 99. Let's visualize two pcs.

In [ ]:
len(X_train)

Visualize two PCs

In [ ]:
# This wasn't that informative
# # Create a dataframe with the first two pcs and the classes
# two_projected = pd.DataFrame(projected_X_train[:, 0:2], columns = ["PC1", "PC2"])
# two_projected["Descriptor"] = y_train.reset_index(drop = True)["Crash Descriptor"]

# # Plot (this takes a while to run so I commented it out, also with so much data it is not very helpful)
# # sns.scatterplot(data = two_projected, x = "PC1", y = "PC2", hue = "Descriptor")

### Look at feature importance

In [ ]:
# Get a list of feature names
feature_names = list(Xtransformer.get_feature_names_out())

# Let's look at the first three PCs to get 95% of the variance
fix, axs = plt.subplots(3, 1, figsize = (6, 28))

for i in range(3):
    pdat = pd.Series(pca.components_[:, i], index = feature_names)
    sns.barplot(x = pdat.values, y = pdat.index, ax = axs[i])
    axs[i].set_title(f"PC{i + 1}")

Surprisingly, it looks like whether the road surface conditions were unknown is one of the most important features. The month, and if there is a curve at the hilcrest also look fairly important.